In [220]:
import pandas as pd
import yfinance as yf
from scipy.stats import norm
import numpy as np
import joblib

In [221]:
#S: current price, K: strike price, T: ttm, r: risk-free, sigma: volatility

# Black-Scholes Pricing Functions
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [222]:
params = joblib.load('../data/params.pkl')

S = params['current_price']
K_range = params['strike_range']
sigma_range = params['vol_range']
r = params['risk_free_rate']
T = params['ttm']

In [223]:
call_matrix = np.zeros((len(sigma_range), len(K_range)))
put_matrix = np.zeros((len(sigma_range), len(K_range)))

for i, sigma in enumerate(sigma_range):
    for j, K in enumerate(K_range):
        call_matrix[i, j] = black_scholes_call(S, K, T, r, sigma)
        put_matrix[i, j] = black_scholes_put(S, K, T, r, sigma)

In [224]:
call_df = pd.DataFrame(call_matrix, index=sigma_range, columns=K_range)
put_df = pd.DataFrame(put_matrix, index=sigma_range, columns=K_range)

In [225]:
joblib.dump({'calls': call_df, 'puts': put_df}, "../data/bs_matrices.pkl")
print("Black-Scholes matrices saved to data/bs_matrices.pkl")


Black-Scholes matrices saved to data/bs_matrices.pkl


In [226]:
call_df.tail()
put_df.tail()

,161.90,170.90,179.89,188.89,197.88,206.88,215.87,224.87,233.86,242.86
0.36,4.555743,6.709300,9.436191,12.768202,16.702547,21.233877,26.322927,31.940714,38.027459,44.545840
0.42,6.737630,9.294527,12.376310,15.996109,20.138607,24.794494,29.926302,35.512213,41.502911,47.873043
0.48,9.120563,12.019398,15.396602,19.255400,23.574214,28.342764,33.525567,39.105868,45.041106,51.313565
0.54,11.643741,14.836943,18.466484,22.530348,27.003822,31.876954,37.116131,42.708387,48.615834,54.825985
0.60,14.265614,17.716381,21.566318,25.810743,30.423441,35.395358,40.694550,46.311020,52.210274,58.383612
